In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('dataincode') \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

#### Rollback no Nível da Tabela
As tabelas Iceberg são imutáveis, significando que os dados nunca são modificados no local. Em vez disso, novas versões dos dados são criadas com cada transação, preservando o histórico completo da tabela. Essa imutabilidade permite reverter facilmente para qualquer ponto anterior no tempo, efetivamente executando um rollback.

 Obs: É possivel fazer rollback a nível do **catálogo**, este não iremos aboradar aqui, pois limita-se Project Nessie **_até onde pesquisei_**

**Opções de Rollback a nivel de tabela no Spark:**
    
- rollback_to_snapshot: Reverte uma tabela para um ID de snapshot específico.

- rollback_to_timestamp: Reverte uma tabela para um timestamp específico.

- set_current_snapshot: Define um ID de snapshot arbitrário como o estado atual da tabela, permitindo alternar entre versões, mesmo que não estejam sequencialmente relacionadas.

- cherrypick_snapshot: Cria um novo snapshot incorporando mudanças de outro snapshot sem criar novos arquivos de dados.
```sql
-- Syntax: para Argumentos nomeados
CALL catalog_name.system.procedure_name(arg_name_2 => arg_2, arg_name_1 => arg_1);

-- Syntax: Argumentos posicionais
CALL catalog_name.system.procedure_name(arg_1, arg_2, ... arg_n);
```

**Fonte:**<br>
https://iceberg.apache.org/docs/1.5.1/spark-procedures/#example <br>
Apache Iceberg: The Definitive Guide

In [ ]:
# Importar funções 
from IPython.display import display, HTML

In [ ]:
%run ./Includes/Utils.ipynb

In [ ]:
%run ./Includes/Datasets.ipynb

In [ ]:
spark.sql("USE iceberg")

In [ ]:
init_data_df = create_dataframe(columns_schema, init_data)

In [ ]:
# Escrever tabela no storage
(
    init_data_df
    .writeTo("iceberg.bronze.order")
    .partitionedBy("country")
    .createOrReplace()
)

In [ ]:
## Ativar WAP
spark.sql("""
    ALTER TABLE iceberg.bronze.order
    SET TBLPROPERTIES (
        'write.wap.enabled' = 'true'
    )"""
)

In [ ]:
spark.sql("SELECT count(*) FROM iceberg.bronze.order").toPandas()

In [ ]:
# Criar uma branch de desenvolvimento chamada etl_rollback

spark.sql("ALTER TABLE iceberg.bronze.order CREATE BRANCH etl_rollback")

In [ ]:
## Inserir novos dados

# feb_data_df = create_dataframe(columns_schema, feb_data)

# feb_data_df.writeTo("iceberg.bronze.order").append()

In [ ]:
spark.sql("SELECT * FROM iceberg.bronze.order.history").show()

In [ ]:
## Listar branches da tabela
spark.sql("SELECT * FROM iceberg.bronze.order.refs;").show()

In [ ]:
## Inserir novos dados na branch etl_rollback

mar_data_df = create_dataframe(columns_schema, mar_data)

mar_data_df.writeTo("iceberg.bronze.order.branch_etl_rollback").append()

In [ ]:
spark.sql("SELECT count(*) FROM iceberg.bronze.order.branch_etl_rollback").toPandas()

In [ ]:
3248082128628347586

spark.sql("CALL iceberg.system.rollback_to_snapshot('iceberg.bronze.order', 3248082128628347586)")

In [ ]:
## Fast forward
spark.sql(f" CALL iceberg.system.fast_forward('iceberg.bronze.order', 'main', 'etl_rollback')")

In [ ]:
# spark.sql("DROP TABLE iceberg.bronze.order PURGE")

In [ ]:
spark.stop()